# Data cleaning project with Pandas

In [1]:
import pandas as pd

## Exploring

In [3]:
#Import the excel file (note : you need the openpyxl module in the environment you are currently using). 
df=pd.read_excel('5_New_markets.xlsx')

In [4]:
#Check if there are full duplicates : answer is no 
len(df.drop_duplicates())

2627

In [5]:
df
# The table has 2627 rows (individuals)
# The table has 10 columns : ID, gender (F/M), ever_married (Y/N), age (num in yrs), graduated (Yes/No), profession (text), 
# work_experience (in years), family_size (number of individuals), spending_score (Low/Med/High)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6
...,...,...,...,...,...,...,...,...,...,...
2622,467954,Male,No,29,No,Healthcare,9.0,Low,4.0,Cat_6
2623,467958,Female,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6
2624,467960,Female,No,53,Yes,Entertainment,NaN,Low,2.0,Cat_6
2625,467961,Male,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4


In [6]:
df.describe()
# We use describe to find out more about our numeric values : 
# AGE mean is 43, median 41, range is from 18 to 89 (adults), with 75% over 30, so mainly active population
# FAMILY SIZE is 1 to 9 individuals (range 8), mean of 2.8 and median of 2.  
# WORK XP ranges from 0 to 14 years, with a mean of 2.5 years and 75% with under 4 years of experience 
    # Work XP is a weird series because older people should have more than 14 years of experience. 
    # For the moment, we are choosing to not use this column. 
    # This can be justified by the fact that age and work_experience are normally redundant. 
    # If we choose to analyse it later, the work XP column could refer the time in the current occupation. 
    # BONUS

,ID,Age,Work_Experience,Family_Size
count,2627.000000,2627.000000,2358.000000,2514.000000
mean,463433.918919,43.649791,2.552587,2.825378
std,2618.245698,16.967015,3.341094,1.551906
min,458989.000000,18.000000,0.000000,1.000000
25%,461162.500000,30.000000,0.000000,2.000000
50%,463379.000000,41.000000,1.000000,2.000000
75%,465696.000000,53.000000,4.000000,4.000000
max,467968.000000,89.000000,14.000000,9.000000


In [8]:
df.describe(include='O')
# We can also use describe to look into our categorical variables : 
# This is useful to see how the data is organized, but also proportions in the whole population. 
# GENDER : male or female (with renaming necessary): 54% male 
# EVER MARRIED (Y/N) : 59% yes
# GRADUATED (Y/N) : 61% yes
# PROFESSION : 
    # note : some missing values 
    # Artists : 30%
    # Heathcare : 16%
    # Entertainment : 12%
    # Doctor : 9%
    # Engineer : 9%
    # Executive : 7%
    # Marketing : 4%
    # Homemaker : 3%
# SPENDING SCORE
    # Low : 61%
    # Average : 24%
    # High : 15% 
# CATEGORY 
    # Category 6 = 64%
    # Category 4 = 15%
    # Category 3 = 10%
    # Category 2 = 5%
    # other categories < 3% 
    # This Var_1 variable is quite weird, we don't really know what it refers to. 
    # For the moment, we are not considering it. If we have time, we'll get back to it to check its correlations with other variables. 
    # BONUS
# Note : these detailed statistics were gathered using value counts for each of the variables.  
        # df["Var_1"].value_counts(normalize=True)
# Note 2: using "normalize" returns a percentage instead of a simple count. 

,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1
count,2627,2577,2603,2589,2627,2595
unique,3,2,2,9,3,7
top,Male,Yes,Yes,Artist,Low,Cat_6
freq,1424,1520,1602,802,1616,1672


In [9]:
df["Var_1"].value_counts(normalize=True)

Cat_6    0.644316
Cat_4    0.148748
Cat_3    0.102890
Cat_2    0.054335
Cat_7    0.025434
Cat_1    0.013102
Cat_5    0.011175
Name: Var_1, dtype: float64

In [10]:
# We noticed there are 3 unique values in gender. When listing them, we notice it is only a upper/lower case issue. 
# So we fixed that. 
df['Gender']=df["Gender"].map({'Male':'M', 'Female':'F', 'female':'F'})
list(df["Gender"].unique())

['F', 'M']

In [11]:
# Then we looked to see if there are missing values
df.isna().sum()
# We notice there are no missing values for gender, age, and spending score, which will be our key variables. 
# There are a few missing variables for ever_married, graduated, and profession (<50 missing values)
# There are a little more missing variables for family size (113/2627 rows)
# And still a little more for work_experience (269/2627 rows) - but we are not considering this column yet. 
# We decided against forcing imputation at this point. 

ID                   0
Gender               0
Ever_Married        50
Age                  0
Graduated           24
Profession          38
Work_Experience    269
Spending_Score       0
Family_Size        113
Var_1               32
dtype: int64

In [12]:
# Before we perform calculations, we checked the data types for the different columns. 
# One thing we can notice is that we mainly have categorical variables (objects). 
df.dtypes

ID                   int64
Gender              object
Ever_Married        object
Age                  int64
Graduated           object
Profession          object
Work_Experience    float64
Spending_Score      object
Family_Size        float64
Var_1               object
dtype: object

In [13]:
# Our initial idea was to try and find a correlation between the different variables and "spending_score" 
# Indeed, we believe this is what the table is trying to show : what are the characteristics of the people with a high spending score ? 
# Answering this question would allow to know which market share (population) the company should focus on for new markets. 

# So we thought of using the correlation method. However, it only works with numeric values. 
# The variable we want to explain, however (spending score) is categorical. 
# So this doesn't work : df.corrwith(df["Var_1"]).value_counts()
# In addition, the numeric values we have don't tell us much about our population 
df.corr(method='pearson', min_periods=1, numeric_only=True)

,ID,Age,Work_Experience,Family_Size
ID,1.000000,-0.022940,-0.025445,0.020830
Age,-0.022940,1.000000,-0.186238,-0.285237
Work_Experience,-0.025445,-0.186238,1.000000,-0.071253
Family_Size,0.020830,-0.285237,-0.071253,1.000000


In [82]:
# Our lovely TA, Berkay, introduces us to the "crosstab" method (https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html) 
# This allows us to check, for some variables, how the population is distributed based on another characteristic. 
# DETAIL OF THE FUNCTION PARAMETERS : pd.crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, 
# margins=False, margins_name='All', dropna=True, normalize=False)
# For instance, spending score per gender (normalized) will return a table which shows that in the total population: 
# 10% are women and in spending score average, 32% are men in spending score low. 

pd.crosstab([df["Spending_Score"]], [df["Gender"]], normalize=True)

Gender,F,M
Spending_Score,,
Average,0.103540,0.134374
High,0.060525,0.086410
Low,0.293871,0.321279


In [71]:
pd.crosstab([df["Spending_Score"]], [df["Gender"], df["Ever_Married"], df["Graduated"]], margins=True, normalize=True)
# if we do it with our currently functional categorical variables
# i.e. df["Gender"], df["Ever_Married"] and df["Graduated"]
# we notice that one key variable is "Ever_Married" : there are no never_married women and no never_married men in the "high" score. 
# Note : we tried including df["Profession"], but the results were hard to read for the moment. 
# In addition, profession and graduated may be redundant. 
# BONUS 

Gender                 F                                       M            \
Ever_Married          No                 Yes                  No             
Graduated             No       Yes        No       Yes        No       Yes   
Spending_Score                                                               
Average         0.000000  0.000000  0.024276  0.078700  0.000000  0.000000   
High            0.000000  0.000000  0.021143  0.038763  0.000000  0.000000   
Low             0.091229  0.122944  0.024276  0.054816  0.114722  0.081832   
All             0.091229  0.122944  0.069695  0.172279  0.114722  0.081832   

Gender                                   All  
Ever_Married         Yes                      
Graduated             No       Yes            
Spending_Score                                
Average         0.038763  0.094753  0.236492  
High            0.031715  0.054033  0.145654  
Low             0.036413  0.091621  0.617854  
All             0.106891  0.240407  1.000000

In [86]:
key_variables= pd.DataFrame(pd.crosstab([df["Spending_Score"]], [df["Gender"], df["Ever_Married"], df["Graduated"]], margins=True, normalize=True))

key_variables
key_variables.to_csv("key_variables.csv", sep=";", index=True)

In [ ]:
# We are here changing the order of our exploration. 
# For our first try, we built a database with only the "high" spending score individuals 
# and we explored it to see who was in there (gender, ever married, graduated, and professions)

# We noticed that : 
# In the high spending category, we only have married people
# 37% men + graduated
# 27% women + graduated 
# 22% men - not graduated 
# 15% women - not graduated 
# So at the moment, we know we should focus on married + graduated
# Gender : not really representative because same proportion as original population 

# Top spending professions : lawyers (31%), executives (30%), and then artists (20%).
# Which sort of repeats the graduate argument, but also pushes towards considering men as key targets (executive is mainly male)

#After doing this, we felt we could go further by adding the age and family size variables to our cross examination
#To do this, however, we needed to change these numeric variables into categorical ones. 
#So here, we are going to do this but in the original dataframe, before creating a dataframe with only the high spending scores. 

In [22]:
# Note : age did not have missing values, but imputation was needed for family size.
# However, family size is oddly distributed, with a very high concentration at 2 people families
# We therefore chose to use the mode (2) instead of the mean to fill the missing values. 
# This means, however, that our "bins" will not have the same size in the end. 
df["Family_Size"].value_counts()

2.0    768
1.0    512
3.0    455
4.0    444
5.0    200
6.0     78
7.0     26
9.0     16
8.0     15
Name: Family_Size, dtype: int64

In [ ]:
#Below is the code we used to explore the "high_spending" category
# df_high=df[df["Spending_Score"].isin(["High"])]
# pd.crosstab([df_high["Spending_Score"]], [df_high["Gender"], df_high["Ever_Married"], df_high["Graduated"], normalize=True)
# df_high["Profession"].value_counts(normalize=True)

In [23]:
# So going back to our original dataframe.
df.describe()

,ID,Age,Work_Experience,Family_Size
count,2627.000000,2627.000000,2358.000000,2514.000000
mean,463433.918919,43.649791,2.552587,2.825378
std,2618.245698,16.967015,3.341094,1.551906
min,458989.000000,18.000000,0.000000,1.000000
25%,461162.500000,30.000000,0.000000,2.000000
50%,463379.000000,41.000000,1.000000,2.000000
75%,465696.000000,53.000000,4.000000,4.000000
max,467968.000000,89.000000,14.000000,9.000000


In [25]:
# We calculate the quartiles in the age category 
list_quart=list (df["Age"].quantile([0.25, 0.5, 0.75]))
list_quart

[30.0, 41.0, 53.0]

In [26]:
# And use it to create a new age_category variable 
df["Age_Category"]="None"
df.loc[df["Age"] <= list_quart[0] , "Age_Category"] = "Under 30"
df.loc[df["Age"] > list_quart[0] , "Age_Category"] = "Between 30 and 41"
df.loc[df["Age"] > list_quart[1] , "Age_Category"] = "Between 41 and 53"
df.loc[df["Age"] >= list_quart[2] , "Age_Category"] = "Over 53"
df

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Age_Category
0,458989,F,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,Between 30 and 41
1,458994,M,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,Between 30 and 41
2,458996,F,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,Over 53
3,459000,M,Yes,59,No,Executive,11.0,High,2.0,Cat_6,Over 53
4,459001,F,No,19,No,Marketing,NaN,Low,4.0,Cat_6,Under 30
...,...,...,...,...,...,...,...,...,...,...,...
2622,467954,M,No,29,No,Healthcare,9.0,Low,4.0,Cat_6,Under 30
2623,467958,F,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6,Between 30 and 41
2624,467960,F,No,53,Yes,Entertainment,NaN,Low,2.0,Cat_6,Over 53
2625,467961,M,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4,Between 41 and 53


In [27]:
# Checking the value counts is useful to see if our categories are relatively similar, and if all rows are categorizes
df["Age_Category"].value_counts()

Between 30 and 41    688
Under 30             675
Over 53              661
Between 41 and 53    603
Name: Age_Category, dtype: int64

In [28]:
# Now we fill the family size missing values with 2 - in a new column
df["Family_Size_Imputed"] = df["Family_Size"].fillna(2)
df

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Age_Category,Family_Size_Imputed
0,458989,F,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,Between 30 and 41,1.0
1,458994,M,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,Between 30 and 41,4.0
2,458996,F,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,Over 53,1.0
3,459000,M,Yes,59,No,Executive,11.0,High,2.0,Cat_6,Over 53,2.0
4,459001,F,No,19,No,Marketing,NaN,Low,4.0,Cat_6,Under 30,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2622,467954,M,No,29,No,Healthcare,9.0,Low,4.0,Cat_6,Under 30,4.0
2623,467958,F,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6,Between 30 and 41,1.0
2624,467960,F,No,53,Yes,Entertainment,NaN,Low,2.0,Cat_6,Over 53,2.0
2625,467961,M,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4,Between 41 and 53,5.0


In [30]:
# We check the value counts and notice, as expected, that our "2 people family" category is important. 
df["Family_Size_Imputed"].value_counts()

2.0    881
1.0    512
3.0    455
4.0    444
5.0    200
6.0     78
7.0     26
9.0     16
8.0     15
Name: Family_Size_Imputed, dtype: int64

In [34]:
# Then we proceed to create our categorical variable for family size. 
# We discussed a lot, and came to the conclusion that 
# single-person families should be separated from the other families 
# two-person families, as a mode, should be a category 
# we can merge three and four person families in one category 
# and then we create a 5 and more people category
# Note : it is unclear whether the "people" considered are adults or not.
        # More precisely : 2 people families can be 2 adults, or a monoparental family.
        # So we shouldn't overinterpret this variable. 
df["Family_Size_Cat"]= "None"
df.loc[df["Family_Size_Imputed"] == 1, "Family_Size_Cat"] = "1 person"
df.loc[df["Family_Size_Imputed"] == 2, "Family_Size_Cat"] = "2 people"
df.loc[(df["Family_Size_Imputed"] == 3)|(df["Family_Size_Imputed"] == 4), "Family_Size_Cat"] = "3 or 4 people"
df.loc[df["Family_Size_Imputed"] >= 5, "Family_Size_Cat"] = "5 or more people"
df

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Age_Category,Family_Size_Imputed,Family_Size_Cat
0,458989,F,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,Between 30 and 41,1.0,1 person
1,458994,M,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,Between 30 and 41,4.0,3 or 4 people
2,458996,F,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,Over 53,1.0,1 person
3,459000,M,Yes,59,No,Executive,11.0,High,2.0,Cat_6,Over 53,2.0,2 people
4,459001,F,No,19,No,Marketing,NaN,Low,4.0,Cat_6,Under 30,4.0,3 or 4 people
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,467954,M,No,29,No,Healthcare,9.0,Low,4.0,Cat_6,Under 30,4.0,3 or 4 people
2623,467958,F,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6,Between 30 and 41,1.0,1 person
2624,467960,F,No,53,Yes,Entertainment,NaN,Low,2.0,Cat_6,Over 53,2.0,2 people
2625,467961,M,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4,Between 41 and 53,5.0,5 or more people


In [42]:
df["Family_Size_Cat"].value_counts()
# again value counts

3 or 4 people       899
2 people            881
1 person            512
5 or more people    335
Name: Family_Size_Cat, dtype: int64

In [33]:
pd.crosstab([df["Family_Size_Cat"]], [df["Spending_Score"]], normalize=True)
# and here we are checking whether family size and spending score are related 
# This table is harder to read, but we can see that the distribution in the high category based on family size 
# doesn't seem much different than the general distribution in the high category. 

Spending_Score,Average,High,Low
Family_Size_Cat,,,
5 or more people,0.027788,0.016749,0.082984
One person,0.000761,0.002284,0.191854
Three or four people,0.105443,0.048725,0.188047
Two people,0.103921,0.079178,0.152265


In [43]:
# At this point we feel we may want to move to SQL to create more readable tables. 
# We are simply exporting to csv
# BONUS would be to get to use sqlachemy. 

# Creating a table with our original database - dropping the unnecessary columns 
new_markets_data = df.copy()


In [44]:
new_markets_data

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Age_Category,Family_Size_Imputed,Family_Size_Cat
0,458989,F,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,Between 30 and 41,1.0,1 person
1,458994,M,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,Between 30 and 41,4.0,3 or 4 people
2,458996,F,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,Over 53,1.0,1 person
3,459000,M,Yes,59,No,Executive,11.0,High,2.0,Cat_6,Over 53,2.0,2 people
4,459001,F,No,19,No,Marketing,NaN,Low,4.0,Cat_6,Under 30,4.0,3 or 4 people
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,467954,M,No,29,No,Healthcare,9.0,Low,4.0,Cat_6,Under 30,4.0,3 or 4 people
2623,467958,F,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6,Between 30 and 41,1.0,1 person
2624,467960,F,No,53,Yes,Entertainment,NaN,Low,2.0,Cat_6,Over 53,2.0,2 people
2625,467961,M,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4,Between 41 and 53,5.0,5 or more people


In [88]:
new_markets_data

,ID,Gender,Ever_Married,Age,Graduated,Profession,Spending_Score,Age_Category,Family_Size_Cat
0,458989,F,Yes,36,Yes,Engineer,Low,Between 30 and 41,1 person
1,458994,M,Yes,37,Yes,Healthcare,Average,Between 30 and 41,3 or 4 people
2,458996,F,Yes,69,No,NaN,Low,Over 53,1 person
3,459000,M,Yes,59,No,Executive,High,Over 53,2 people
4,459001,F,No,19,No,Marketing,Low,Under 30,3 or 4 people
...,...,...,...,...,...,...,...,...,...
2622,467954,M,No,29,No,Healthcare,Low,Under 30,3 or 4 people
2623,467958,F,No,35,Yes,Doctor,Low,Between 30 and 41,1 person
2624,467960,F,No,53,Yes,Entertainment,Low,Over 53,2 people
2625,467961,M,Yes,47,Yes,Executive,High,Between 41 and 53,5 or more people


In [89]:
#new_markets_data.drop(columns=["Work_Experience","Family_Size","Family_Size_Imputed", "Age", "Var_1"], inplace=True)
new_markets_data.drop(columns=["Age"], inplace=True)

In [90]:
new_markets_data
new_markets_data.to_csv('new_markets_data_comma.csv', sep=',', index=False)
new_markets_data.to_csv('new_markets_data_semico.csv', sep=';', index=False)

In [92]:
df_high=new_markets_data[new_markets_data["Spending_Score"].isin(["High"])]
df_high.drop(columns=["Spending_Score"], inplace=True)
df_high


/var/folders/5g/jskfzb8j377fh7wl3_rxvkrc0000gn/T/ipykernel_6141/104975496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high.drop(columns=["Spending_Score"], inplace=True)


,ID,Gender,Ever_Married,Graduated,Profession,Age_Category,Family_Size_Cat
3,459000,M,Yes,No,Executive,Over 53,2 people
5,459003,M,Yes,Yes,Doctor,Between 41 and 53,5 or more people
20,459056,M,Yes,No,Executive,Over 53,3 or 4 people
21,459057,M,Yes,No,Lawyer,Over 53,2 people
23,459059,M,Yes,No,Executive,Over 53,2 people
...,...,...,...,...,...,...,...
2591,467868,F,Yes,Yes,Entertainment,Over 53,2 people
2592,467876,F,Yes,Yes,Artist,Between 41 and 53,2 people
2593,467879,F,Yes,Yes,Artist,Between 41 and 53,3 or 4 people
2605,467905,M,Yes,Yes,Executive,Between 30 and 41,3 or 4 people


In [93]:
df_high.to_csv("new_markets_high_comma.csv", sep=",", index=False)
df_high.to_csv("new_markets_high_semico.csv", sep=";", index=False)

In [63]:
crosstabs_high= pd.DataFrame(pd.crosstab([df_high["Spending_Score"]],[df_high["Gender"], df_high["Ever_Married"], df_high["Graduated"], df_high["Age_Category"], df_high["Family_Size_Cat"], df_high["Profession"]], normalize=True))
crosstabs_high.to_csv("cross_comma.csv", sep=",", index=True)
crosstabs_high.to_csv("cross_semico.csv", sep=";", index=True)